# HW11: ML Fairness

In [3]:
import pandas as pd

# Data Prep

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv')
df.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


In [5]:
df.columns

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid'],
      dtype='object')

In [6]:
df['days_from_recid'] = pd.to_datetime(df['r_offense_date']) - pd.to_datetime(df['c_offense_date'])
df['days_from_recid'] = df['days_from_recid'].dt.days
df['two_year_recid2'] = df['days_from_recid']<=730
df['two_year_recid2'] = df['two_year_recid2'].astype(int)

In [7]:
d = pd.get_dummies(df['c_charge_degree'])
df = pd.concat([df, d], axis=1)
df.head()
df = df.rename(columns={'F':'c_charge_degreeF'})
df = df.rename(columns={'M':'c_charge_degreeM'})
d = pd.get_dummies(df['age_cat'])
df = pd.concat([df, d], axis=1)
df = df.rename(columns={'25 - 45':'age_cat_25 - 45'})
df = df.rename(columns={'Greater than 45':'age_cat_Greater than 45'})
df = df.rename(columns={'Less than 25':'age_cat_Less than 25'})
d = pd.get_dummies(df['race'])
df = pd.concat([df, d], axis=1)
d = pd.get_dummies(df['sex'])
df = pd.concat([df, d], axis=1)
d = pd.get_dummies(df['score_text'])
df = pd.concat([df, d], axis=1)
df = df.rename(columns={'High':'score_text_high'})
df = df.rename(columns={'Medium':'score_text_medium'})
df = df.rename(columns={'Low':'score_text_low'})

In [8]:
df['male'] = (df['sex'] == 'Male').astype(int)
df['felony'] = (df['c_charge_degree'] == 'F').astype(int)

In [25]:
# judge decision: "jailed" = more than one day in jail.
df['jail_days'] = pd.to_datetime(df['c_jail_out']) - pd.to_datetime(df['c_jail_in'])
df['jailed'] = (df.jail_days.dt.days > 1).astype(int)
D = df['jailed']
D.value_counts()

0    4283
1    2931
Name: jailed, dtype: int64

In [8]:
# ML outcome
Y = df['two_year_recid2'] 
Y.value_counts()

0    4557
1    2657
Name: two_year_recid2, dtype: int64

In [9]:
# predictors
predictors = ['felony',
              'male', 'age', 
              'juv_fel_count','juv_misd_count', 'juv_other_count', 'priors_count']
X = df[predictors]
X.describe()

,felony,male,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count
count,7214.000000,7214.000000,7214.000000,7214.000000,7214.000000,7214.000000,7214.000000
mean,0.646798,0.806626,34.817993,0.067230,0.090934,0.109371,3.472415
std,0.477998,0.394971,11.888922,0.473972,0.485239,0.501586,4.882538
min,0.000000,0.000000,18.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,25.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,31.000000,0.000000,0.000000,0.000000,2.000000
75%,1.000000,1.000000,42.000000,0.000000,0.000000,0.000000,5.000000
max,1.000000,1.000000,96.000000,20.000000,13.000000,17.000000,38.000000


In [11]:
df['race'].value_counts()

African-American    3696
Caucasian           2454
Hispanic             637
Other                377
Asian                 32
Native American       18
Name: race, dtype: int64

In [23]:
# indicator variable for non-white
NW = (df['race'] != 'Caucasian').astype(int)
NW.describe()

count    7214.000000
mean        0.659828
std         0.473800
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: race, dtype: float64

# Predict Recidivism from Features

Use logistic regression to predict recidivism from defendant features. Use nested training/test split so we can get clean test-set predictions for the whole dataset (see Lecture 6 Slide 9).

In [24]:
# perform nested train/test split 
from sklearn.model_selection import train_test_split
X_a, X_b, Y_a, Y_b, D_a, D_b,NW_a, NW_b = train_test_split(X, Y, D, NW, test_size = 0.5)

# train a logit model to predict recidism (Y) from predictors (X).
# form clean test-set predictions for recidivism in the full dataset
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression()
# TODO

Evaluate test set performance.

In [1]:
# show a confusion matrix
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
# TODO

# show an sklearn classification report
from sklearn.metrics import classification_report
# print(classification_report(Y, Ypred)

# compute ratio of false positives to false negatives.
# TODO

# Compare treatment of whites and non-whites for recidivism prediction

**Refore to Week 11 slides for these concepts.**

Compare base rates for recidivism outcomes of whites and non-whites

In [ ]:
# TODO

Produce confusion matrices, classification reports, and ratio of false positives to false negatives, separately for whites and non-whites.

In [ ]:
# TODO

What concepts of "fairness" (from class) are (approximately) satisfied by this classifier? Explain.

[answer here].

# Predict Judge Decision from Features

Use logistic regression to predict judge decision from defendant features. In a second model, include Y-hat from previous section as a predictor.

In [20]:
# train a logit model to predict judge decision (D) from predictors (X). 
# train a logit model to predict judge decision (D) from predictors (X) and predicted recidivism risk (Yhat)
# form clean test-set predictions for the decisions in the full dataset

# TODO

Produce confusion matrices, classification reports, and ratio of false positives to false negatives, separately for whites and non-whites, and with/without including Y-hat as a predictor.

In [18]:
# TODO

You can use a custom classifier `threshold` for the decision with this type of code snippet: `decisions = (logit.predict_proba(X) >= threshold).astype(int)`

Find group-specific thresholds for both models (with/without Yhat) that obtain statistical parity.

In [ ]:
# TODO

# Bonus 1: More Fairness Constraints

Find the threshold for both models (with/without Yhat) that obtains error rate balance (equality of recalls for each class).

In [ ]:
# TODO

Find the threshold for both models (with/without Yhat) that obtains predictive parity (equality of precisions for each class).

In [26]:
# TODO

Find the threshold for both models (with/without Yhat) that obtains treatment equality (ratio of false positives to false negatives).

In [27]:
# TODO

# Bonus 2: Pre-Processing for Fairness

Regress all predictors in `X` on the protected attribute `NW` and produce residuals `Xtilde`. Re-do the machine learning tasks above and discuss how it changes the fairness metrics.

In [ ]:
# TODO